In [13]:
import pandas as pd
from notebooks.consts import *
from tauso.file_utils import read_human_genome_fasta_dict
from tauso.consts import *
import numpy as np
from tauso.util import get_antisense
import pickle
from tauso.genome.read_human_genome import get_locus_to_data_dict
from tauso.file_utils import read_human_genome_fasta_dict
import RNA
import tauso
from notebooks.preprocessing import *
from src.tauso.new_model.populate.calculate_mfe import calculate_mfe_features
from notebooks.visualization.paper_utils import *
from notebooks.features.feature_extraction import save_feature

In [2]:
csv_path = NOTEBOOK_PATH / 'data' / 'data_asoptimizer_updated.csv'
all_data = pd.read_csv(str(csv_path), low_memory=False)

Preprocessing ASO Experimental Dataset

In [3]:
aso_data = preprocess_aso_data(csv_path=csv_path)

Preprocessing complete. Final valid rows: 29987


In [4]:
aso_sample = aso_data.head(5).copy()

In [11]:
aso_sample

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,log_inhibition,sense_sequence,pre_mrna_sequence,sense_start,sense_length
0,0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,...,41212.0,0.901972,16,0,A431,-4.204842,TAGCATTTGTTTTAGC,GATTTTCCTAGGCGGCGGCCGCGGCGGCGGAGGCAGCAGCGGCGGC...,41212,16
1,1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,...,23686.0,0.518395,16,0,A431,-4.532707,GATATTCACCATTATA,GATTTTCCTAGGCGGCGGCCGCGGCGGCGGAGGCAGCAGCGGCGGC...,23686,16
2,2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,...,43363.0,0.949049,16,1,A431,-3.637849,CCAGAAATCTTCATGC,GATTTTCCTAGGCGGCGGCCGCGGCGGCGGAGGCAGCAGCGGCGGC...,43363,16
3,3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,...,23680.0,0.518264,16,0,A431,-4.276805,TTTGAAGATATTCACC,GATTTTCCTAGGCGGCGGCCGCGGCGGCGGAGGCAGCAGCGGCGGC...,23680,16
4,4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,...,41168.0,0.901009,16,0,A431,-4.159039,CATACTAGTACAAGTG,GATTTTCCTAGGCGGCGGCCGCGGCGGCGGAGGCAGCAGCGGCGGC...,41168,16


In [5]:
calculate_mfe_features(aso_sample)

Starting MFE population with Flank=120, Window=45, Step=7


,index,mfe_win45_flank120_step7
0,0,-0.075053
1,1,-0.128003
2,2,-0.098254
3,3,-0.115271
4,4,-0.133168


In [7]:
df_mfe = calculate_mfe_features(aso_data)

Starting MFE population with Flank=120, Window=45, Step=7


In [9]:
print(df_mfe.shape)
df_mfe.head(20)

(29987, 2)


,index,mfe_win45_flank120_step7
0,0,-0.075053
1,1,-0.128003
2,2,-0.098254
3,3,-0.115271
4,4,-0.133168
5,5,-0.120503
6,6,-0.143433
7,7,-0.073915
8,8,-0.067864
9,9,-0.056720


In [11]:
merged_df = aso_data.merge(df_mfe,"inner",on="index")

In [14]:
save_feature(df=df_mfe, feature_name= 'mfe_win45_flank120_step7')

ViennaRNA energy calculation

### Final best parameters
Parameters that were tested:

- **Step sizes:**
  `[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`

- **Windows sizes:**
  `[20, 30, ,35, 40, 45, 50, 60, 70]`

The chosen parameters are: window size = 45, step = 7